# Image selection - Felix

## Before running the notebook:
Download the **last** 5 json files from:
https://www.dropbox.com/sh/08ccnkpr5wdihdg/AAAz_s0IeX99co_34sARF70ma/OSM%20dataset%20JSON?dl=0&preview=osm_parking_polygons_09.json.gzip&subfolder_nav_tracking=1

And put them in a subfolder "ndata"

## 1) Load packages and data

In [2]:
import os, io, json
import pandas as pd
import geopandas as gpd
from ast import literal_eval
from shapely.geometry import shape, mapping as shapely_mapping
from shapely import wkt
import folium
from PIL import Image, ImageDraw
from __future__ import print_function
from ipywidgets import interact,interact_manual,HBox,Output,Tab
import time

import cv2
import random
import re
import sys
import multiprocessing.dummy as mp 
import numpy as np

In [26]:
# Functions needed: 
def parking_name(row_or_pt):
    """
    create an id from a (lat,lon) point or an object containing the pt property
    """
    # no "pt" item => assume it is a tuple
    try:
        pt = row_or_pt
        a = round(pt[0], 7)
        b = round(pt[1], 7)
    except:
        return np.nan
    
    return f"{a}_{b}_ts2" ## FLAG


def find_tags(df2):
    ls = []
    for j in range(0, len(df2)):
        index = True
        for i in df2.iloc[j,:].all_tags:
            if i["key"] == "surface" and i["value"] == "asphalt":
            #if i["key"] == "name" and i["value"].lower().__contains__("truck"):   # or i["value"].lower().__contains__("lkw")):   # FLAG
                ls.append(True)
                index = False
        
        if index:
            ls.append(False)
        
    return ls


def get_point(geom):
    try:
        pt = re.split(r",", geom[9:])[0]
        pt = tuple(map(float, pt.split(' ')))
        pt_s = (pt[1], pt[0])
    except:
        return np.nan
    
    return pt_s

In [3]:
# Pre-processing
# THIS TAKES A FEW MINUTES

start_time = time.localtime()
start_t = time.time()
print("Started at:", time.asctime(start_time))

#file_nr = [5,6,7,8,9]
file_nr = [7,8]
first = True

for i in file_nr:
    df = pd.read_json(f"ndata/osm_parking_polygons_0{i}.json", lines=True)
    df = df[find_tags(df)]
    df["pt"] = df["geometry"].transform(lambda x: get_point(x))
    df["id"] = [parking_name(x) for x in df["pt"]]
    df['geometry'] = df['geometry'].apply(wkt.loads)
    df = df[["id", "pt", "geometry", "all_tags"]]
    df = df.dropna()
    
    if first:
        temp = df
        first = False
    else: 
        temp = pd.concat([temp, df])
    
    print(f"File nr. {i} successfully loaded")
        
real_polys = gpd.GeoDataFrame(temp, geometry="geometry")

end_t = time.time()
print("Computation time (Min.):", (end_t-start_t)/60)

Started at: Mon May 16 16:44:21 2022
File nr. 7 successfully loaded
File nr. 8 successfully loaded
Computation time (Min.): 1.2988752762476603


In [27]:
"""
df = pd.read_csv("ndata/osm_data_cars.csv", index_col=0)
df["pt"] = df["geometry"].transform(lambda x: get_point(x))
df["id"] = [parking_name(x) for x in df["pt"]]
df['geometry'] = df['geometry'].apply(wkt.loads)
real_polys = gpd.GeoDataFrame(df, geometry="geometry")
"""

In [28]:
real_polys = real_polys.reset_index()

In [6]:
# Backup
#real_polys_backup = real_polys.copy()

In [29]:
# Final dataframe
real_polys #= real_polys.loc[:249535,]

,index,id_new,id,pt,geometry,all_tags
0,0,47.006201_6.9670316_ts2,47.006201_6.9670316_ts2,"(47.006201, 6.9670316)","POLYGON ((6.96703 47.00620, 6.96687 47.00635, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
1,1,59.1875955_17.6167656_ts2,59.1875955_17.6167656_ts2,"(59.1875955, 17.6167656)","POLYGON ((17.61677 59.18760, 17.61681 59.18755...","[{'key': 'access', 'value': 'restricted'}, {'k..."
2,2,51.9754365_8.5019413_ts2,51.9754365_8.5019413_ts2,"(51.9754365, 8.5019413)","POLYGON ((8.50194 51.97544, 8.50216 51.97518, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
3,3,39.8643349_-4.0193694_ts2,39.8643349_-4.0193694_ts2,"(39.8643349, -4.0193694)","POLYGON ((-4.01937 39.86433, -4.01786 39.86549...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
4,4,43.7565471_-79.2317921_ts2,43.7565471_-79.2317921_ts2,"(43.7565471, -79.2317921)","POLYGON ((-79.23179 43.75655, -79.23185 43.756...","[{'key': 'access', 'value': 'customers'}, {'ke..."
...,...,...,...,...,...,...
1256,1256,5.345678_-3.949021_ts2,5.345678_-3.949021_ts2,"(5.345678, -3.949021)","POLYGON ((-3.94902 5.34568, -3.94897 5.34568, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
1257,1257,45.172425_5.7538404_ts2,45.172425_5.7538404_ts2,"(45.172425, 5.7538404)","POLYGON ((5.75384 45.17242, 5.75421 45.17224, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
1258,1258,44.1196918_4.8438315_ts2,44.1196918_4.8438315_ts2,"(44.1196918, 4.8438315)","POLYGON ((4.84383 44.11969, 4.84390 44.11967, ...","[{'key': 'amenity', 'value': 'parking'}]"
1259,1259,53.6895026_-1.3077646_ts2,53.6895026_-1.3077646_ts2,"(53.6895026, -1.3077646)","POLYGON ((-1.30776 53.68950, -1.30737 53.68929...","[{'key': 'access', 'value': 'yes'}, {'key': 'a..."


## 2) Load functions to get image with labels

In [24]:
# Create image
BOUNDS_RGB = (0xff,0x78, 0x00)

def rgbcolor(r,g,b):
    """
    turn r, g, b integers into a CSS color code 
    """
    return f"#{r:02x}{g:02x}{b:02x}"

def geometry_bbox(geometry, init_bounds=None, x_offset=0., y_offset=0.):
    """
    return a bounding box (southwest, northeast) of a given geometry
    optionally add a margin
    """
    bounds = init_bounds or [[1000,-1000],[-1000,1000]]
    xs = [c[0] for c in geometry.exterior.coords]
    
    ys = [c[1] for c in geometry.exterior.coords]
    
    return [
        #southwest
        [min(bounds[0][0], min(ys)-y_offset), max(bounds[0][1], max(xs)+x_offset)],
        #northeast
        [max(bounds[1][0], max(ys)+y_offset), min(bounds[1][1], min(xs)-x_offset)]
    ]


def make_map(pt, geometry=None, show_geometry=False, show_bounds=False, size=320, seednr=100):
    m = folium.Map(location=pt, min_zoom=15, width=size, height=size, zoom_control=False, attribution_control=False)
    
    folium.TileLayer(
            #tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',  # Original tile server
            tiles = 'https://clarity.maptiles.arcgis.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', # New tile server
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = False,
            control = True
    ).add_to(m)
    
    # Default bounds
    bounds = [[pt[0], pt[1]],[pt[0], pt[1]]] #southwest, northeast

    # Lines of geometry and Box around them ("Bounds")
    if geometry:
        if show_geometry:
            folium.GeoJson(data=geometry).add_to(m)

        bounds = geometry_bbox(geometry, init_bounds=bounds, x_offset=0, y_offset=0)
    
    # Plot box around geometry
    if show_bounds:
        folium.Rectangle(bounds=bounds, color=rgbcolor(*BOUNDS_RGB), fill=True, fill_color=rgbcolor(*BOUNDS_RGB), fill_opacity=0).add_to(m) 
    
    # Create random offset
    random.seed(seednr)
    x_offset_l = random.randint(1,9)/9200
    random.seed(seednr+10)
    x_offset_r = random.randint(1,9)/9200
    random.seed(seednr+20)
    y_offset_l = random.randint(1,9)/9200
    random.seed(seednr+30)
    y_offset_r = random.randint(1,9)/9200
    
    crop_bounds = [[bounds[0][0]-y_offset_l, bounds[0][1]+x_offset_l],[bounds[1][0]+y_offset_r, bounds[1][1]-x_offset_r]] #southwest, northeast
    
    # Create red random crop box
    if show_bounds:
        folium.Rectangle(bounds=crop_bounds, color="red", fill=False, fill_color=rgbcolor(*BOUNDS_RGB), fill_opacity=0.).add_to(m) 
    
    #m.fit_bounds(bounds, max_zoom = 18)
    m.fit_bounds(crop_bounds) # FLAG
    
    return m, bounds

## 3) Manually filter images

### Instructions

- Start with the selection procedure by executing the cell below.

- If an image is fine, just press enter while the courser is in the input cell.

- If it is faulty in any way, enter any character of your choice.

- To stop the procedure, type and enter "exit".

- If the image shown is a truck parking lot, type "t" (pay attention to not press t to blacklist an item!)

- The counter and id of the last image checked will be printed. If you stop and want to proceed later on, change the start_nr variable below to the last number printed.

- If the interactive "pop-up" of the map is too big (i.e. you always have to scroll down to get to the input bar), just adjust the window width (in jupyter lab, increase the width of the data browser on the left)

<br>

Caution:
- Don't forget to save the blacklist and whitelist as csv!

- Always change the file name after saving a new set of lists.

- Always note the last counter value you checked when exiting, in order to know where to resume.

- If you exit the loop, the last image shown will **not** yet be saved as ok nor faulty.

In [ ]:
row = real_polys.loc["51.087_13.274_ts2"]
m, b = make_map(row.pt, geometry=row.geometry, show_geometry=True, show_bounds=False, size=800, seednr=95) # create map with boundaries
m

In [30]:
# Sart selection

start_nr = 1255 # if you stop somewhere inbetween, enter here the last image nr. printed

blacklist = []
whitelist = []
trucklist = []

for i in range(start_nr, real_polys.shape[0]):
    row = real_polys.iloc[i]
    m, b = make_map(row.pt, geometry=row.geometry, show_geometry=True, show_bounds=True, size=640, seednr=95) # create map with boundaries
    
    display(m)
    inp = input(f"Image Nr. {i} ok? Just enter. If truck, type \"t\". To exit type \"exit\"")
    
    if inp == "exit":
        print("Image to be checked next:", row["id"])
        break
    elif inp == "t":
        trucklist.append(row["id"])
    elif inp != "":
        blacklist.append(row["id"])
    else: 
        whitelist.append(row["id"])
    
    print("Last image checked:", row["id"])
    print("")

Image Nr. 1255 ok? Just enter. If truck, type "t". To exit type "exit" exit


Image to be checked next: 49.3715495_6.1522216_ts2


In [ ]:
# Reasons for high dropout: No satellite imagery available, very small images, clustering of parking lots (often, there are several parking lots close to each other)

In [ ]:
# Blacklist: 36.253389_139.6935681_ts2, 47.1308546_-0.1473121_ts2, 45.2955723_-75.6065874_ts2

In [10]:
# Print blacklist
blacklist

233

In [17]:
trucklist

['45.8561953_3.8128949_ts2',
 '46.8795844_11.4365074_ts2',
 '52.2429238_12.1415345_ts2',
 '39.6121_-85.8708025_ts2']

In [11]:
# Print whitelist (to record which images have already been checked)
whitelist

22

## Save blacklist and whitelist

In [14]:
def savefile(list1, csv_name, PATH):
    if os.path.isfile(f"{PATH}/{csv_name}.csv"):
        return f"File \"{csv_name}\" already created! Change filename!"
    else:
        dict1 = {'id': list1}
        df = pd.DataFrame(dict1)
        df.to_csv(f"{PATH}/{csv_name}.csv") 
        return f"File \"{csv_name}\" successfully saved!"

In [18]:
# Save lists as csv:
path = "ndata/cars"

# saving lists
print(savefile(blacklist, "blacklist5", path))
print(savefile(whitelist, "whitelist5", path))
#print(savefile(trucklist, "trucklist2", path))

File "blacklist5" successfully saved!
File "whitelist5" successfully saved!
File "trucklist2" successfully saved!


### Notes on first Truck iteration

In [ ]:
# Difficult: 
d = ["53.244_-3.191_ts2", "43.296_3.216_ts2", "41.501_-74.206_ts2", "42.571_-113.782_ts2", "41.247_-87.861_ts2", "49.96_7.951_ts2"]

In [ ]:
# Example, Truck Parking lot not built yet: 51.963_6.029_ts2, -37.805_144.753_ts2, 59.303_15.261_ts2, 41.092_29.288_ts2, 51.196_13.739_ts2
# Not fully built yet ? 42.898_-74.099_ts2
# Ngeative example: -29.487_152.335_ts2

In [ ]:
# "The perfect" truck parking lot: 51.087_13.274_ts2

In [ ]:
# Nice car parking lot with feldern around: 45.0139306_0.1170134_ts2